In [ ]:
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import plot_importance
from sklearn.feature_selection import SelectKBest
from sklearn.inspection import partial_dependence
from sklearn.inspection import plot_partial_dependence
#path="../../data/"

from utils import *
from MLModels import *
from DataProcessing import *


In [ ]:
path="../../../ukb/data/"

drpvars1='death|AD|Unknown|answered_sexual_history_questions|speciality_of_consultant|APOE4|\
hospital_episode_type|history_of_psychiatric|samesex|year_of_birth|sexual_partners|using_computer|sex_inference|\
age_first_had|age_started|age_when|ageing|drive_faster'

drpvarsapoe='AD|Unknown|answered_sexual_history_questions|speciality_of_consultant|APOE4|\
hospital_episode_type|history_of_psychiatric|samesex|year_of_birth|Genotype|age'


drvarsdef=['DIAB','DIAB_bef','CERVASC','CERVASC_bef','PD','CERVASC','EPIL']

genos=[
 'Genotype_e1/e4',
 'Genotype_e2/e2',
 'Genotype_e2/e3',
 'Genotype_e2/e4',
 'Genotype_e3/e3',
 'Genotype_e3/e4',
 'Genotype_e4/e4']

genotypes=['Genotype_e1/e2',"Genotype_e2/e2","Genotype_e2/e3","Genotype_e3/e3","Genotype_e2/e4","Genotype_e1/e4",
          "Genotype_e3/e4","Genotype_e4/e4"]

gen_lkup={'Genotype_e2/e2':1,"Genotype_e1/e2":2,"Genotype_e2/e3":3,"Genotype_e3/e3":4,
          "Genotype_e2/e4":5,"Genotype_e1/e4":6,"Genotype_e3/e4":7,"Genotype_e4/e4":8}

gen_lkup_rev={1:'Genotype_e2/e2',2:"Genotype_e1/e2",3:"Genotype_e2/e3",4:"Genotype_e3/e3",
          5:"Genotype_e2/e4",6:"Genotype_e1/e4",7:"Genotype_e3/e4",8:"Genotype_e4/e4"}


In [ ]:

config = dict(scale_pos_weight = 6,subsample = 1, min_child_weight = 5, max_depth = 5, gamma= 2, 
              colsample_bytree= 0.6,smote=1)

config_rebal = dict(scale_pos_weight = 1,subsample = 1, min_child_weight = 5, max_depth = 5, gamma= 2, 
              colsample_bytree= 0.6,smote=1)


mod_xgb=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
           colsample_bynode=1, learning_rate=0.1,
           max_delta_step=0,  missing=None,
           n_estimators=60, n_jobs=4, nthread=4, objective='binary:logistic',
           random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=config['scale_pos_weight'],
           min_child_weight=config['min_child_weight'],
           gamma=config['gamma'], colsample_bytree=config['colsample_bytree'],max_depth=config['max_depth'],
           seed=42, silent=None, subsample=1, verbosity=1)

mod_xgb_rebal=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
           colsample_bynode=1, learning_rate=0.1,
           max_delta_step=0,  missing=None,
           n_estimators=60, n_jobs=4, nthread=4, objective='binary:logistic',
           random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=config_rebal['scale_pos_weight'],
           min_child_weight=config_rebal['min_child_weight'],
           gamma=config_rebal['gamma'], colsample_bytree=config_rebal['colsample_bytree'],max_depth=config_rebal['max_depth'],
           seed=42, silent=None, subsample=1, verbosity=1)

mod_rf = RandomForestClassifier(max_depth=5, random_state=0)

In [ ]:
def traintest_df(df,rat=0.7):
    maskAD=(df['AD']==1)
    
    sizeAD=len(df[maskAD])
    sizenoAD=len(df[~maskAD])
    sizeAD_tr=round(rat*sizeAD)
    sizenoAD_tr=round(rat*sizenoAD)
    
    train=pd.concat([df[maskAD].sample(sizeAD_tr),df[~maskAD].sample(sizenoAD_tr)],axis=0)
    test=df[~(df['eid'].isin(train['eid']))]
    return train,test

def score_df(df_train,df_val,dropcols,model,dropcolyn=1):
    if dropcolyn!=1:
        dropcols='AD'
        
    X_train=df_train.drop(columns=dropcols)
    y_train=df_train['AD']
    
    model_train=model.fit(X_train,y_train)
    
    X_test=df_val.drop(columns=dropcols)
    df_valout=df_val.copy()
    
    df_valout['risk']=model.predict_proba(X_test)[:, 1]
    return df_valout


       
def int_vars(df,var1,vars=genos):
    for var in vars:
        df[str(var1)+" "+var]=df[var]*df[var1]
    return df

def newfeats(df,shapsum,depvar='AD',feats=30):
    
    if feats==max:
        cols=[col for col in df if col in np.asarray(shapsum['column_name'])]
    
    else:
        cols=[col for col in df if col in np.asarray(shapsum.head(feats)['column_name'])]
    
    if 'eid' in cols:
        df_out=df[np.append(['AD'],cols)]   
    else:
        df_out=df[np.append(['AD','eid'],cols)]
    return df_out

def cleandf(df,alm=1):
    df=replacenullsmean(df)
    
    if alm==1:
        df['AST_ALT_ratio']=round((df['aspartate_aminotransferase_f30650']\
                                          /df['alanine_aminotransferase_f30620']),1)

    #change to any of the pollution metrics in the top 40%
        
        mask_sleep_mr=(df['sleep_duration_f1160'].between(6,8))
        mask_sleep_high=(df['sleep_duration_f1160']>8)
        
        df['sleep']=0
        df['sleep'][mask_sleep_mr]=1
        df['sleep'][mask_sleep_high]=2
        
        #pollution
        df['polluted']=0
        df['polluted'][(df['particulate_matter_air_pollution_pm25_absorbance_2010_f24007']>9)]=1


    df=col_spec_chars(df)
    #df=replace_genotype(df,gen_lkup,genotypes)
    return df

def maskapoedf(df,apoe=1):
    apoemask=(df['Genotype_e3/e4']==1)|(df['Genotype_e4/e4']==1)|\
    (df['Genotype_e2/e4']==1)|(df['Genotype_e1/e4']==1)
    non_apoemask=(df['Genotype_e2/e3']==1)|(df['Genotype_e3/e3']==1)|\
    (df['Genotype_e1/e2']==1)|(df['Genotype_e2/e2']==1)

    if apoe==2:
        return df[apoemask|non_apoemask]
    elif apoe==1:  
        return df[apoemask]
    elif apoe==0:  
        return df[non_apoemask]
    

def summarise_score_data(df):
    df=pd.DataFrame(df.groupby(['eid']).agg({'AD':'max','risk':['mean','std']})).reset_index()
    df.columns=['eid','AD','risk','std_risk']   
    return df

def runmodel(df,dropcols,reps,splits,model):
    
    dropcols=[col for col in dropcols if 'AD' not in dropcols]
    df_out=df.drop(columns=dropcols)
    df_test_out=pd.DataFrame([])
    for reps in range(reps):
        if reps-round(reps/5)*5==0:
            print(reps)
        kf = KFold(n_splits=splits,shuffle=True)

        for train_index, test_index in kf.split(df_out):
            df_train, df_test = df_out.iloc[train_index,: ], df_out.iloc[test_index, :]
            
            df_score=df_test[['eid','AD']]

            X_train, X_test = df_train.drop(columns='AD'), df_test.drop(columns='AD')
            y_train, y_test = df_train['AD'], df_test['AD']

            model.fit(X_train,y_train)   
            #plot_importance(model,max_num_features=10)
            
            boruta_selector = BorutaPy(model, n_estimators='auto', verbose = 2)

            boruta_selector.fit(np.array(X_train), np.array(y_train))
            
            df_score['risk']=model.predict_proba(X_test)[:, 1]
            df_test_out=pd.concat([df_test_out,df_score])
    df_test_out=summarise_score_data(df_test_out)
    print(df_test_out.shape)
    return df_test_out




def plot_ROCAUC_mult(y_test, y_score,y_test1,y_score1,y_test2,y_score2,title,l1,l2,l3):
    
    
    fpr, tpr, _ = roc_curve(y_test,y_score)
    fpr1, tpr1, _ = roc_curve(y_test1,y_score1)
    fpr2, tpr2, _ = roc_curve(y_test2,y_score2)
    
    colors = ['darksalmon', 'gold', 'royalblue', 'mediumseagreen', 'violet'] 
    
    mean_auc = auc(fpr, tpr)
    mean_auc1 = auc(fpr1, tpr1)
    mean_auc2 = auc(fpr2, tpr2)
   
    
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_test, y_score))


    plt.plot(fpr, tpr, 'b', alpha = 0.8,
             label=r'%s (AUC = %0.2f)' % (l1,mean_auc))
    plt.plot(fpr1, tpr1, 'c', alpha = 0.8,
             label=r'%s apoe (AUC = %0.2f)' % (l2,mean_auc1))
    plt.plot(fpr2, tpr2, 'm', alpha = 0.8,
             label=r'%s non apoe (AUC = %0.2f)' % (l3,mean_auc2))

    plt.plot([0, 1], [0, 1], linestyle = '--', lw = 2, color = 'r', label = 'Luck', alpha= 0.8)
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc="lower right")
    plt.title(title)
    #plt.axes().set_aspect('equal', 'datalim')
    plt.show()

In [ ]:
AD_model_full70=pd.read_pickle('%s%s' % (path,'AD_model_full70.p'))

In [ ]:
AD_model_full70=cleandf(AD_model_full70,alm=1)
AD_model_full_gfilt=maskapoedf(AD_model_full70,apoe=2)
AD_model_full_gfilt_apoe=maskapoedf(AD_model_full_gfilt,apoe=1)
AD_model_full_gfilt_nonapoe=maskapoedf(AD_model_full_gfilt,apoe=0)

In [ ]:
varimp_full,score_g=newmodelrun(AD_model_full_gfilt,dropvars=dropvars_gfilt,model=mod_xgb,depvar='AD',
                    reps=20,splits=5,rebalance=0,max_display=15,plot_type="dot",shap=1,plots=1)
varimp_full_apoe_pos,score_apoe_g=newmodelrun(AD_model_full_gfilt_apoe,dropvars=dropvars_gfilt,model=mod_xgb,depvar='AD',
                    reps=20,splits=3,rebalance=0,max_display=15,plot_type="dot",shap=1,plots=1)
varimp_full_apoe_neg,score_napoe_g=newmodelrun(AD_model_full_gfilt_nonapoe,dropvars=dropvars_gfilt,model=mod_xgb,depvar='AD',
                    reps=20,splits=3,rebalance=0,max_display=15,plot_type="dot",shap=1,plots=1)

In [ ]:
varimp_full.to_csv('%s%s' % (path,'varimp_full_.csv'))
varimp_full_apoe_pos.to_csv('%s%s' % (path,'vvarimp_full_apoe_pos_.csv'))
varimp_full_apoe_neg.to_csv('%s%s' % (path,'varimp_full_apoe_neg_.csv'))

In [ ]:
plot_ROCAUC_mult(score_g['actual'], score_g['mean_score'],score_apoe_g['actual'],
                 score_apoe_g['mean_score'],
                 score_napoe_g['actual'],
                 score_napoe_g['mean_score'],title='AUC ROC Plot for Models 1-3:  Modifiable and non-modifiable features',
                 l1='Model 1',l2='Model 2',l3='Model 3')